Importing Modulus

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model,model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

Loading the preprocessed data

In [2]:
import numpy as np
from keras.utils import np_utils
X=np.load('drive/My Drive/data1.npy')
tar=np.load('drive/My Drive/target1.npy')
y=np_utils.to_categorical(tar)

Using TensorFlow backend.


Data Augmentation

In [0]:
#splitting the dataset
(trainX, testX, trainY, testY) = train_test_split(X,y,
	test_size=0.20, stratify=y, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

Training the Model

In [4]:
INIT_LR = 1e-4 #initial learning rate
EPOCHS = 20 #number of epochs
BS = 32 #batch_size
baseModel = NASNetLarge(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224,224,3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

343613440/343610240 [==============================] - 3s 0us/step
[INFO] compiling model...


In [5]:
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

[INFO] training head...
Epoch 1/20
23/23 [==============================] - 18s 783ms/step - loss: 0.4383 - accuracy: 0.7792 - val_loss: 0.2319 - val_accuracy: 0.9255
Epoch 2/20
23/23 [==============================] - 11s 480ms/step - loss: 0.2711 - accuracy: 0.8958 - val_loss: 0.1794 - val_accuracy: 0.9415
Epoch 3/20
23/23 [==============================] - 11s 489ms/step - loss: 0.1789 - accuracy: 0.9375 - val_loss: 0.1525 - val_accuracy: 0.9521
Epoch 4/20
23/23 [==============================] - 11s 479ms/step - loss: 0.1846 - accuracy: 0.9181 - val_loss: 0.1396 - val_accuracy: 0.9574
Epoch 5/20
23/23 [==============================] - 11s 482ms/step - loss: 0.1469 - accuracy: 0.9444 - val_loss: 0.1319 - val_accuracy: 0.9574
Epoch 6/20
23/23 [==============================] - 11s 480ms/step - loss: 0.1569 - accuracy: 0.9458 - val_loss: 0.1244 - val_accuracy: 0.9628
Epoch 7/20
23/23 [==============================] - 11s 482ms/step - loss: 0.1636 - accuracy: 0.9306 - val_loss: 0.122

Saving the model and obtaining classification Report

In [6]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs))

[INFO] evaluating network...
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        99
           1       0.97      0.97      0.97        89

    accuracy                           0.97       188
   macro avg       0.97      0.97      0.97       188
weighted avg       0.97      0.97      0.97       188

